In [1]:
setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/data")

library(data.table)

source("/mnt/lareaulab/reliscu/code/analyzeSyntheticDatasets/makeSyntheticDatasets_0.51.r")

Here I make pseudobulk samples, where each sample contains a mix of cell types

In [2]:
counts <- fread("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/scRNA-seq/tasic_2018/ALM/tasic_2018_ALM_STAR_gene_counts.csv", data.table=FALSE)
cell_meta <- fread("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/scRNA-seq/tasic_2018/ALM/tasic_2018_ALM_STAR_sampleinfo.csv", data.table=FALSE)

In [5]:
sort(table(cell_meta$cell_subclass))


        L4       Peri         CR      Meis2        SMC       VLMC   Serpinf1 
         3          4         11         11         55         72         78 
Macrophage       Endo      Oligo        L6b       Sncg      Astro    L2/3 IT 
        85         96         98        145        148        215        325 
     L6 CT      L5 PT      L6 IT         NP      Pvalb      Lamp5        Sst 
       350        368        394        542        896        913       1139 
       Vip      L5 IT 
      1224       2401 

In [7]:
all.equal(colnames(counts)[-1], cell_meta$Cell_ID)

[1] TRUE

In [ ]:
cell_frac <- data.frame(table(cell_meta$cell_subclass) / nrow(cell_meta))
cell_meta$Freq <- cell_frac$Freq[match(cell_meta$cell_subclass, as.character(cell_frac[,1]))]

cell.name = which(colnames(cell_meta) == "Cell_ID")
cell.type = which(colnames(cell_meta) == "cell_subclass")
cell.frac = which(colnames(cell_meta) == "Freq")

In [ ]:
set.seed(1)

# Note: this takes several hours

makeSyntheticDatasets(
    expr=counts,
    sampleindex=c(2:ncol(counts)),
    cell.info=cell_meta,
    cell.name=cell.name,
    cell.type=cell.type,
    cell.frac=cell.frac,
    pcnt.cells=20,
    pcnt.var=10,
    no.samples=200,
    no.datasets=1,
    with.replacement=TRUE
)

In [ ]:
# # Debugging

# expr <- counts
# sampleindex=c(2:ncol(counts))
# cell.info=cell_meta
# cell.name=cell.name
# cell.type=cell.type
# cell.frac=cell.frac
# pcnt.cells=20
# pcnt.var=0.3
# no.samples=200
# no.datasets=1
# i=1
# with.replacement=TRUE

# ## Check to ensure rows of cell.info and columns of expr are in same order:
# if(all.equal(colnames(expr[,sampleindex]),as.character(cell.info[,cell.name]))!=TRUE){
#     stop("Cell names in expr and cell.info do not match!")
# }

# no.cells=round(length(sampleindex)*pcnt.cells*.01)
# meta=min(sampleindex)-1
# print(paste("Building dataset ",i,"...",sep=""))

# if(is.null(cell.type)){
    
#     dat.legend=matrix(nrow=length(cell.info[,1]),ncol=no.samples+1,data=0)
#     dat.legend[,1]=as.character(cell.info[,cell.name])
#     colnames(dat.legend)=c("Cell.name",paste("Sample",c(1:no.samples),sep=""))
#     leg.meta=1
    
# } else {
    
#     dat.legend=matrix(nrow=length(cell.info[,1]),ncol=no.samples+2,data=0)
#     dat.legend[,1]=as.character(cell.info[,cell.name])
#     dat.legend[,2]=as.character(cell.info[,cell.type])
#     colnames(dat.legend)=c("Cell.name","Cell.type",paste("Sample",c(1:no.samples),sep=""))
#     leg.meta=2
    
# }

# dat.synth=matrix(nrow=nrow(expr),ncol=no.samples,data=-88)
# colnames(dat.synth)=paste("Sample",c(1:no.samples),sep="")

# for(j in 1:no.samples){
    
#     if(!is.null(cell.frac)){
        
#         all.ct=unique(as.character(cell.info[,cell.type]))
#         sample.vec=c()
        
#         for(k in 1:length(all.ct)){
            
#             rest1=is.element(cell.info[,cell.type],all.ct[k])
#             base.cells=round(no.cells*unique(cell.info[rest1,cell.frac]))
            
#             if(pcnt.var==0){
                
#                     sample.vec=c(sample.vec,(sample(c(1:nrow(cell.info))[rest1],base.cells,replace=with.replacement)))
                    
#             } else {
                
#                 act.cell.range=c(round(base.cells-(base.cells*pcnt.var*.01)):round(base.cells+(base.cells*pcnt.var*.01)))
#                 act.cells=sample(act.cell.range[act.cell.range>=0&act.cell.range<length(sampleindex)],1)
#                 sample.vec=c(sample.vec,sample(c(1:nrow(cell.info))[rest1],act.cells,replace=with.replacement))
#             }
        
#         sample.vec=sort(sample.vec)
#         #Check: table(cell.info$Cell.Class[sample.vec])
#         dat.synth[,j]=apply(as.data.frame(expr[,sample.vec+meta]),1,sum,na.rm=T)
#         dat.legend[sample.vec,j+leg.meta]=1
        
#         } ## end of for(k in 1:length(all.ct)){

#     }   
    
#     # } else {
    
#     # if(pcnt.var==0){
        
#     #     sample.vec=sort(sample(c(1:nrow(cell.info)),no.cells,replace=with.replacement))
#     #     dat.synth[,j]=apply(expr[,sample.vec+meta],1,sum,na.rm=T)
#     #     dat.legend[sample.vec,j+leg.meta]=1
        
#     # } else {

#     #     act.cell.range=c(round(no.cells-(no.cells*pcnt.var*.01)):round(no.cells+(no.cells*pcnt.var*.01)))
#     #     act.cells=sample(act.cell.range[act.cell.range>=0&act.cell.range<length(sampleindex)],1)
#     #     sample.vec=sort(sample(c(1:nrow(cell.info)),act.cells,replace=with.replacement))
#     #     dat.synth[,j]=apply(expr[,sample.vec+meta],1,sum,na.rm=T)
#     #     dat.legend[sample.vec,j+leg.meta]=1
    
#     #     }
    
#     # }
    
#     print(j)
    
# } ## end of for(j in 1:no.samples)
